In [73]:
# Thanks for Moorissa Tjokro I get some of my codes from her explanation to deal with purchase data


##https://medium.datadriveninvestor.com/how-to-build-a-recommendation-system-for-purchase-data-step-by-step-d6d7a78800b6
    

#  Data Prep & Filtering

In [72]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from pandas import Series, DataFrame
import time
import turicreate as tc
from sklearn.model_selection import train_test_split
from functools import reduce
import random

In [2]:

file1 = "Data1.csv"
file2 = "Data2.csv"


In [3]:
def pre_process_df(filename):
    if filename:
        df = pd.read_csv(filename)
        df = df.drop_duplicates()
        df = df[df.brand.notnull()]
        return df[df.price > 0]
    return None
  
def gen_purchase_count(files):
    df = pd.concat([pre_process_df(f) for f in files])
    df = df.loc[:, ['event_type', 'brand', 'user_id']]
    df['purchase_count'] = [1 if e == 'purchase' else 0 for e in df['event_type']]
    df = df.drop(['event_type'], axis = 1)
    df = df.groupby(['brand', 'user_id']).sum()
    df = df[df.purchase_count != 0]
    return df.reset_index()

In [4]:
purchase_count = gen_purchase_count([file1,file2])

In [5]:
purchase_count

,brand,user_id,purchase_count
0,airnails,10280338,1
1,airnails,105697630,1
2,airnails,126051562,2
3,airnails,149936015,1
4,airnails,160905182,2
...,...,...,...
151238,zinger,579632103,1
151239,zinger,579681799,10
151240,zinger,579750441,1
151241,zinger,579849574,1


In [6]:
user_id  = np.unique(purchase_count['user_id'])
num_brand_purchased = np.zeros(len(user_id))
for loc, user in enumerate(user_id):
    num_brand_purchased[loc] = np.sum(purchase_count['user_id'] == user)

In [7]:
num_brand_purchased

array([2., 2., 3., ..., 2., 1., 1.])

In [8]:
np.sum(purchase_count['user_id'] == 579681799)

3

In [9]:
purchase_count[purchase_count['user_id'] == 579681799]

,brand,user_id,purchase_count
72519,irisk,579681799,1
89748,lovely,579681799,2
151239,zinger,579681799,10


In [10]:
user_id

array([  9794320,  10079204,  10280338, ..., 579924768, 579925377,
       579944216])

In [11]:
idx = np.where(num_brand_purchased>=5)

In [12]:
np.where(num_brand_purchased>=5)

(array([    6,     9,    11, ..., 46949, 46951, 46953]),)

In [13]:
filitered_user_id = user_id[idx]

In [14]:
filtered_data = purchase_count.loc[purchase_count['user_id'].isin(filitered_user_id)]

In [15]:
filtered_data

,brand,user_id,purchase_count
1,airnails,105697630,1
2,airnails,126051562,2
3,airnails,149936015,1
5,airnails,169938179,2
6,airnails,170386254,4
...,...,...,...
151231,zinger,579242176,2
151234,zinger,579501173,1
151236,zinger,579523833,1
151237,zinger,579608040,1


In [16]:
filtered_data=filtered_data[['user_id','brand','purchase_count']]

In [17]:
filtered_data.columns=['customerId','productId','purchase_count']

In [18]:
data=filtered_data

In [19]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(data)

In [20]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')

In [21]:
df_matrix

productId,airnails,almea,andrea,ardell,art-visage,artex,aura,balbcare,barbie,batiste,...,uskusi,veraclara,vilenta,vosev,weaver,yoko,ypsed,yu-r,zeitun,zinger
customerId,,,,,,,,,,,,,,,,,,,,,
29025780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33609704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34236465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36180886,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40821287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579811378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
579841776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
579847583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

In [23]:
d = df_matrix_norm.reset_index() 
d.index.names = ['scaled_purchase_freq'] 
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()


In [24]:
print(data_norm.shape)
data_norm.head()

(75607, 3)


,customerId,productId,scaled_purchase_freq
27,105697630,airnails,0.000000
36,126051562,airnails,0.021277
49,149936015,airnails,0.000000
65,169938179,airnails,0.021277
67,170386254,airnails,0.063830


In [25]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [26]:
train_data, test_data = split_data(data)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [27]:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(filtered_data[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

In [30]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
            model = tc.item_similarity_recommender.create(train_data, 
                                                        user_id=user_id, 
                                                        item_id=item_id, 
                                                        target=target, 
                                                    similarity_type='pearson')
    elif name == 'jaccard':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='jaccard')

    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model


In [31]:
name = 'popularity'
target = 'purchase_count'
popularity = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 60633 observations with 10609 users and 210 items.

Data prepared in: 0.120788s

60633 observations to process; with 210 unique items.

recommendations finished on 1000/75792 queries. users per second: 48412.1

...

recommendations finished on 71000/75792 queries. users per second: 215045

recommendations finished on 72000/75792 queries. users per second: 216381

recommendations finished on 73000/75792 queries. users per second: 217036

recommendations finished on 74000/75792 queries. users per second: 217140

recommendations finished on 75000/75792 queries. users per second: 217293

In [32]:
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 60633 observations with 10608 users and 212 items.

Data prepared in: 0.109495s

...

recommendations finished on 71000/75792 queries. users per second: 252465

recommendations finished on 72000/75792 queries. users per second: 253074

recommendations finished on 73000/75792 queries. users per second: 253752

recommendations finished on 74000/75792 queries. users per second: 252585

recommendations finished on 75000/75792 queries. users per second: 250106

In [33]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 60485 observations with 10605 users and 172 items.

Data prepared in: 0.086707s

60485 observations to process; with 172 unique items.

recommendations finished on 1000/75792 queries. users per second: 48341.9

...

recommendations finished on 71000/75792 queries. users per second: 237330

recommendations finished on 72000/75792 queries. users per second: 238467

recommendations finished on 73000/75792 queries. users per second: 238981

recommendations finished on 74000/75792 queries. users per second: 239533

recommendations finished on 75000/75792 queries. users per second: 238390

In [34]:
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 60633 observations with 10609 users and 210 items.

Data prepared in: 0.13242s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 71000/75792 queries. users per second: 145172

recommendations finished on 72000/75792 queries. users per second: 145357

recommendations finished on 73000/75792 queries. users per second: 145759

recommendations finished on 74000/75792 queries. users per second: 145930

recommendations finished on 75000/75792 queries. users per second: 145469

In [35]:
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 60633 observations with 10608 users and 212 items.

Data prepared in: 0.082772s

...

recommendations finished on 71000/75792 queries. users per second: 147264

recommendations finished on 72000/75792 queries. users per second: 146562

recommendations finished on 73000/75792 queries. users per second: 147005

recommendations finished on 74000/75792 queries. users per second: 146655

recommendations finished on 75000/75792 queries. users per second: 146350

In [36]:
name = 'cosine' 
target = 'scaled_purchase_freq' 
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 60485 observations with 10605 users and 172 items.

Data prepared in: 0.090875s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 71000/75792 queries. users per second: 153269

recommendations finished on 72000/75792 queries. users per second: 153666

recommendations finished on 73000/75792 queries. users per second: 153971

recommendations finished on 74000/75792 queries. users per second: 154178

recommendations finished on 75000/75792 queries. users per second: 154230

In [37]:
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 60633 observations with 10609 users and 210 items.

Data prepared in: 0.123109s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 71000/75792 queries. users per second: 140926

recommendations finished on 72000/75792 queries. users per second: 141272

recommendations finished on 73000/75792 queries. users per second: 141737

recommendations finished on 74000/75792 queries. users per second: 142327

recommendations finished on 75000/75792 queries. users per second: 141544

In [38]:
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 60633 observations with 10608 users and 212 items.

Data prepared in: 0.093988s

...

recommendations finished on 71000/75792 queries. users per second: 192884

recommendations finished on 72000/75792 queries. users per second: 191682

recommendations finished on 73000/75792 queries. users per second: 191618

recommendations finished on 74000/75792 queries. users per second: 190934

recommendations finished on 75000/75792 queries. users per second: 187991

In [39]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 60485 observations with 10605 users and 172 items.

Data prepared in: 0.103337s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 71000/75792 queries. users per second: 131823

recommendations finished on 72000/75792 queries. users per second: 132496

recommendations finished on 73000/75792 queries. users per second: 132743

recommendations finished on 74000/75792 queries. users per second: 132584

recommendations finished on 75000/75792 queries. users per second: 130827

In [40]:
name = 'jaccard'
target = 'purchase_count'
jacc = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 60633 observations with 10609 users and 210 items.

Data prepared in: 0.083468s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 71000/75792 queries. users per second: 148596

recommendations finished on 72000/75792 queries. users per second: 148841

recommendations finished on 73000/75792 queries. users per second: 147590

recommendations finished on 74000/75792 queries. users per second: 146053

recommendations finished on 75000/75792 queries. users per second: 143976

In [41]:
name = 'jaccard'
target = 'purchase_dummy'
jacc_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 60633 observations with 10608 users and 212 items.

Data prepared in: 0.150522s

...

recommendations finished on 71000/75792 queries. users per second: 141368

recommendations finished on 72000/75792 queries. users per second: 142045

recommendations finished on 73000/75792 queries. users per second: 142534

recommendations finished on 74000/75792 queries. users per second: 142539

recommendations finished on 75000/75792 queries. users per second: 141296

In [42]:
name = 'jaccard'
target = 'scaled_purchase_freq'
jacc_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 60485 observations with 10605 users and 172 items.

Data prepared in: 0.094137s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 71000/75792 queries. users per second: 157077

recommendations finished on 72000/75792 queries. users per second: 156204

recommendations finished on 73000/75792 queries. users per second: 155842

recommendations finished on 74000/75792 queries. users per second: 153778

recommendations finished on 75000/75792 queries. users per second: 151761

In [43]:
models_w_counts = [popularity, cos, pear,jacc]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy,jacc_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm,jacc_norm]
names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts','Jaccard Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy','Jaccard Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts','Jaccard Similarity on Scaled Purchase Counts']

In [45]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/8199 queries. users per second: 64512

recommendations finished on 2000/8199 queries. users per second: 74985

recommendations finished on 3000/8199 queries. users per second: 77259.9

recommendations finished on 4000/8199 queries. users per second: 80970

...

recommendations finished on 5000/8171 queries. users per second: 60743

recommendations finished on 6000/8171 queries. users per second: 61970

recommendations finished on 7000/8171 queries. users per second: 62279.8

recommendations finished on 8000/8171 queries. users per second: 62559.1


Precision and recall summary statistics by cutoff
+--------+---------------------+---------------------+
| cutoff |    mean_precision   |     mean_recall     |
+--------+---------------------+---------------------+
|   1    |  0.2746297882756089 | 0.15847426924100533 |
|   2    | 0.21539591237302655 | 0.24380076266626236 |
|   3    | 0.18153632766287275 | 0.30470979441423685 |
|   4    |  0.1595275976012729 |  0.3547669555318552 |
|   5    | 0.14022763431648524 | 0.38680530058880375 |
|   6    |  0.1274017868070004 | 0.41864001219955155 |
|   7    | 0.11619490532720231 | 0.44418729226280346 |
|   8    | 0.10691775792436667 | 0.46704285189782635 |
|   9    | 0.09963420769931608 | 0.48943087535671087 |
|   10   | 0.09336678497123987 |  0.5087666796821121 |
+--------+---------------------+---------------------+
[10 rows x 3 columns]


Overall RMSE: 0.10190272227802054

Per User RMSE (best)
+------------+------+-------+
| customerId | rmse | count |
+------------+------+-------+
| 4784213

In [ ]:
eval_counts

In [46]:
final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy', similarity_type='cosine')
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 75792 observations with 10609 users and 213 items.

Data prepared in: 0.158766s

...

recommendations finished on 71000/75792 queries. users per second: 130331

recommendations finished on 72000/75792 queries. users per second: 130787

recommendations finished on 73000/75792 queries. users per second: 130863

recommendations finished on 74000/75792 queries. users per second: 130457

recommendations finished on 75000/75792 queries. users per second: 127897

In [47]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

(757920, 4)


,customerId,productId,score,rank
0,105697630,ingarden,0.260868,1
1,105697630,grattol,0.250795,2
2,105697630,freedecor,0.236701,3
3,105697630,domix,0.206590,4
4,105697630,milv,0.204796,5


In [ ]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=5)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: ' , '.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')

In [64]:
recomendation = final_model.recommend(users=users_to_recommend, k=5)
df_rec = recomendation.to_dataframe()
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
    .transform(lambda x: ' , '.join(x.astype(str)))
df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
    .sort_values('customerId').set_index('customerId')

recommendations finished on 1000/75792 queries. users per second: 70141

recommendations finished on 2000/75792 queries. users per second: 99735.7

recommendations finished on 3000/75792 queries. users per second: 113611

recommendations finished on 4000/75792 queries. users per second: 119080

recommendations finished on 5000/75792 queries. users per second: 128710

...

recommendations finished on 71000/75792 queries. users per second: 158011

recommendations finished on 72000/75792 queries. users per second: 158540

recommendations finished on 73000/75792 queries. users per second: 157865

recommendations finished on 74000/75792 queries. users per second: 157580

recommendations finished on 75000/75792 queries. users per second: 157111

In [65]:
df_output=df_output.reset_index()

In [66]:
df_output.head()

,customerId,recommendedProducts
0,29025780,"bpw.style , ingarden , grattol , masura , uno ..."
1,33609704,"irisk , bpw.style , ingarden , grattol , uno ,..."
2,34236465,"bpw.style , grattol , masura , domix , freedec..."
3,36180886,"irisk , bpw.style , grattol , masura , freedec..."
4,40821287,"bpw.style , ingarden , masura , domix , freede..."


## This is for Streamlit

In [67]:
# new=recomendation.to_dataframe()

# new.columns=["user_id","brand","score","rank"]
# new.to_csv('new_brand.csv',index=False)

In [48]:
def create_output(model, users_to_recommend, n_rec, print_csv=True):
    recomendation = model.recommend(users=users_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['customerId', 'recommendedProducts']].drop_duplicates() \
        .sort_values('customerId').set_index('customerId')
    if print_csv:
        df_output.to_csv('option1_recommendation.csv')
        print("An output file can be found in 'output' folder with name 'option1_recommendation.csv'")
    return df_output

In [49]:
df_output = create_output(pear_norm, users_to_recommend, 5, print_csv=True)
print(df_output.shape)
df_output.head()

recommendations finished on 1000/75792 queries. users per second: 32180.2

recommendations finished on 2000/75792 queries. users per second: 57002.8

recommendations finished on 3000/75792 queries. users per second: 77019.8

recommendations finished on 4000/75792 queries. users per second: 92142.5

recommendations finished on 5000/75792 queries. users per second: 105033

...

recommendations finished on 73000/75792 queries. users per second: 173704

recommendations finished on 74000/75792 queries. users per second: 172449

recommendations finished on 75000/75792 queries. users per second: 171347

An output file can be found in 'output' folder with name 'option1_recommendation.csv'
(10609, 1)


,recommendedProducts
customerId,
29025780,labay|dewal|osmo|coxir|sanoto|labay|dewal|osmo...
33609704,labay|dewal|osmo|mielle|coxir|labay|dewal|osmo...
34236465,labay|dewal|osmo|coxir|sanoto|labay|dewal|osmo...
36180886,labay|osmo|dewal|mielle|coxir|labay|osmo|dewal...
40821287,labay|dewal|osmo|coxir|sanoto|labay|dewal|osmo...


In [50]:
def customer_recomendation(customer_id):
    if customer_id not in df_output.index:
        print('Customer not found.')
        return customer_id
    return df_output.loc[customer_id]

In [51]:
customer_recomendation(40821287)

recommendedProducts    labay|dewal|osmo|coxir|sanoto|labay|dewal|osmo...
Name: 40821287, dtype: object